In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
data_pos = []
data_neg = []
with open("./../train_pos.txt") as f:
    for i in f: 
        t = i.replace('<user>', '')
        t1 = t.replace('<url>', '')
        data_pos.append(t1)

with open("./../train_neg.txt") as f:
    for i in f:
        t = i.replace('<user>', '')
        t1 = t.replace('<url>', '')
        data_neg.append(t1)

In [3]:
data_pos_ls = [tweet.split(" ") for tweet in data_pos]
data_neg_ls = [tweet.split(" ") for tweet in data_neg]
pd_tweet_pos = pd.Series(data_pos_ls)
pd_tweet_neg = pd.Series(data_neg_ls)

In [8]:
import re, string
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

personlised_remove_reg_tokens = ["\n", "..."]
personlised_remove_tokens = ["rt"]

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        

        """
        need to look into this
        """
        
        # for rm_token in personlised_remove_tokens:
        #     token = re.sub(rm_token,"", token)

        # for rm_token in personlised_remove_reg_tokens:
        #      pat = r'(\w*%s\w*)' % rm_token
        #      token = re.sub(pat,"", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in pd_tweet_pos:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in pd_tweet_neg:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [10]:
pos_data_labels = ['pos'] * len(positive_cleaned_tokens_list)
neg_data_labels = ['neg'] * len(negative_cleaned_tokens_list)
data = positive_cleaned_tokens_list + negative_cleaned_tokens_list
data_labels = pos_data_labels + neg_data_labels

In [11]:
N = len(data)
np_data = np.array(data)
print(np_data.shape)
np_data_labels = np.array(data_labels)
np.random.seed(20)
shuffler = np.random.permutation(N)
Ntrain = 150000

(200000,)


/var/folders/rv/89dfs8q53tx_7htbblgttb100000gn/T/ipykernel_60192/1476114094.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_data = np.array(data)


In [14]:
X_train = np_data[shuffler[:Ntrain]]
y_train = np_data_labels[shuffler[:Ntrain]]
X_test = np_data[shuffler[Ntrain:]]
y_test = np_data_labels[shuffler[Ntrain:]]

In [35]:
def custom_join(x):
    ss = ""
    for idx,s in enumerate(x):
        if idx == len(x)-1:
            ss += s
        else:
            ss += (s + " ")
    return ss

In [ ]:
tst = X_train[:10]
print(tst)
tst = pd.Series(tst)
tst1 = tst.apply(custom_join)
print(tst1)

In [43]:
X_train = pd.Series(X_train).apply(custom_join)
X_test = pd.Series(X_test).apply(custom_join)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer(lowercase=False)
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [46]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [47]:
clf.fit(tf_x_train,y_train)

LinearSVC(random_state=0)

In [48]:
y_test_pred = clf.predict(tf_x_test)

In [49]:
from sklearn.metrics import classification_report

report=classification_report(y_test, y_test_pred,output_dict=True)

In [50]:
print(report)

{'neg': {'precision': 0.7985080812266888, 'recall': 0.7726051565820602, 'f1-score': 0.7853430883042246, 'support': 24939}, 'pos': {'precision': 0.7807885581754929, 'recall': 0.8059933761621643, 'f1-score': 0.7931907875360781, 'support': 25061}, 'accuracy': 0.78934, 'macro avg': {'precision': 0.7896483197010908, 'recall': 0.7892992663721122, 'f1-score': 0.7892669379201513, 'support': 50000}, 'weighted avg': {'precision': 0.7896267018829685, 'recall': 0.78934, 'f1-score': 0.7892765121132143, 'support': 50000}}


In [51]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(max_iter=1000, solver="saga")

In [52]:
clf1.fit(tf_x_train,y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [53]:
y_test_pred=clf1.predict(tf_x_test)

In [54]:
report1=classification_report(y_test, y_test_pred,output_dict=True)
print(report1)

{'neg': {'precision': 0.8042244771174156, 'recall': 0.7786198323910342, 'f1-score': 0.7912150598973189, 'support': 24939}, 'pos': {'precision': 0.7864629665441888, 'recall': 0.8113802322333507, 'f1-score': 0.7987273155786, 'support': 25061}, 'accuracy': 0.79504, 'macro avg': {'precision': 0.7953437218308022, 'recall': 0.7950000323121924, 'f1-score': 0.7949711877379595, 'support': 50000}, 'weighted avg': {'precision': 0.7953220527879029, 'recall': 0.79504, 'f1-score': 0.7949803526898905, 'support': 50000}}
